In [10]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import polars as pl
import datetime 
from tqdm import tqdm

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# from metric import score # Import event detection ap score function
# These are variables to be used by the score function
column_names = {
    'series_id_column_name': 'series_id',
    'time_column_name': 'step',
    'event_column_name': 'event',
    'score_column_name': 'score',
}

tolerances = {
    'onset': [12, 36, 60, 90, 120, 150, 180, 240, 300, 360], 
    'wakeup': [12, 36, 60, 90, 120, 150, 180, 240, 300, 360]
}

In [2]:
pip install plotly

^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install plotly

In [8]:
!pip install metric

  Using cached metric-0.10.0-py3-none-any.whl (44 kB)
  Using cached numexpr-2.6.8.tar.gz (94 kB)

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Shubham\anaconda3\python.exe' 'C:\Users\Shubham\anaconda3\lib\site-packages\pip' install --ignore-installed --no-user --prefix 'C:\Users\Shubham\AppData\Local\Temp\pip-build-env-3zxgt1rb\overlay' --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- wheel setuptools 'Cython>=0.29.2' 'numpy==1.13.3; python_version=='"'"'3.5'"'"'' 'numpy==1.13.3; python_version=='"'"'3.6'"'"'' 'numpy==1.14.5; python_version>='"'"'3.7'"'"''
       cwd: None
  Complete output (577 lines):
  Ignoring numpy: markers 'python_version == "3.5"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.6"' don't match your environment
    Using cached wheel-0.41.2-py3-none-any.whl (64 kB)
    Using cached setuptools-68.2.2-py3-none-any.whl (807 kB)
    Using cached Cython-3.0.2-cp38-cp38-win_amd64.whl (2.8 MB)
    Using cached numpy-1.14.5.zip (4.9 MB)


  Using cached scipy-1.3.0.tar.gz (23.6 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'
  Using cached metric-0.9.0-py3-none-any.whl (45 kB)
  Using cached metric-0.8.0-py3-none-any.whl (45 kB)
  Using cached metric-0.7.0-py3-none-any.whl (45 kB)
  Using cached metric-0.6.0-py3-none-any.whl (45 kB)
  Using cached metric-0.5.0-py3-none-any.whl (45 kB)
  Using cached metric-0.3.0-py3-none-any.whl (8.1 kB)

The conflict is caused by:
    metric 0.10.0 depends on scipy==1.3.0
    metric 0.9.0 depends on scipy==1.3.0
    metric 0.8.0 depends on scipy==1.3.0
    metric 0.7.0 depends on scipy==1.3.0
    metric 0.6.0 depends on scipy==1.3.0
    metric 0.5.0 depends on scipy==1.3.0
    metric 0.3.0 depends on scipy==1.3.0

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip attempt to solve the dependency conflict




    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\Shubham\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Shubham\\AppData\\Local\\Temp\\pip-install-35kvdhty\\numpy_026d09186a1747a9b898b657716bc7e3\\setup.py'"'"'; __file__='"'"'C:\\Users\\Shubham\\AppData\\Local\\Temp\\pip-install-35kvdhty\\numpy_026d09186a1747a9b898b657716bc7e3\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Shubham\AppData\Local\Temp\pip-wheel-vsb7i_ls'
         cwd: C:\Users\Shubham\AppData\Local\Temp\pip-install-35kvdhty\numpy_026d09186a1747a9b898b657716bc7e3\
    Complete output (264 lines):
    Running from numpy source directory.
    C:\Users\Shubham\AppData\Local\Temp\pip-install-35kvdhty\numpy_026d09186a1747a9b898b657716bc7e3\numpy\distutils\misc_util.py:464: SyntaxWarning: "is" with a lite

In [12]:
# Importing data 

# Column transformations

dt_transforms = [
    pl.col('timestamp').str.to_datetime(), 
    (pl.col('timestamp').str.to_datetime().dt.year()-2000).cast(pl.UInt8).alias('year'), 
    pl.col('timestamp').str.to_datetime().dt.month().cast(pl.UInt8).alias('month'),
    pl.col('timestamp').str.to_datetime().dt.day().cast(pl.UInt8).alias('day'), 
    pl.col('timestamp').str.to_datetime().dt.hour().cast(pl.UInt8).alias('hour')
]

data_transforms = [
    pl.col('anglez').cast(pl.Int16), # Casting anglez to 16 bit integer
    (pl.col('enmo')*1000).cast(pl.UInt16), # Convert enmo to 16 bit uint
]

train_series = pl.scan_parquet('train_series.parquet').with_columns(
    dt_transforms + data_transforms
    )

train_events = pl.read_csv('train_events.csv').with_columns(
    dt_transforms
    )

test_series = pl.scan_parquet('test_series.parquet').with_columns(
    dt_transforms + data_transforms
    )

# Getting series ids as a list for convenience
series_ids = train_events['series_id'].unique(maintain_order=True).to_list()

# Removing series with mismatched counts: 
count_mismatches = train_events.drop_nulls().group_by(['series_id', 'night']).agg([
    ((pl.col('event') == 'onset').sum() == (pl.col('event') == 'wakeup').sum()).alias('balanced')
    ]).sort(by=['series_id', 'night']).filter(pl.col('balanced')==False)['series_id'].to_list()

train_events = train_events.filter(~pl.col('series_id').is_in(count_mismatches))

# Updating list of series ids, not including series with no non-null values.
series_ids = train_events.drop_nulls()['series_id'].unique(maintain_order=True).to_list()

train_series = train_series.filter(pl.col('series_id').is_in(series_ids))

In [13]:
features, feature_cols = [pl.col('hour')], ['hour']

for mins in [5, 30, 60*2, 60*8] :
    features += [
        pl.col('enmo').rolling_mean(12 * mins, center=True, min_periods=1).abs().cast(pl.UInt16).alias(f'enmo_{mins}m_mean'),
        pl.col('enmo').rolling_max(12 * mins, center=True, min_periods=1).abs().cast(pl.UInt16).alias(f'enmo_{mins}m_max')
    ]

    feature_cols += [ 
        f'enmo_{mins}m_mean', f'enmo_{mins}m_max'
    ]

    # Getting first variations
    for var in ['enmo', 'anglez'] :
        features += [
            (pl.col(var).diff().abs().rolling_mean(12 * mins, center=True, min_periods=1)*10).abs().cast(pl.UInt32).alias(f'{var}_1v_{mins}m_mean'),
            (pl.col(var).diff().abs().rolling_max(12 * mins, center=True, min_periods=1)*10).abs().cast(pl.UInt32).alias(f'{var}_1v_{mins}m_max')
        ]

        feature_cols += [ 
            f'{var}_1v_{mins}m_mean', f'{var}_1v_{mins}m_max'
        ]

id_cols = ['series_id', 'step', 'timestamp']

train_series = train_series.with_columns(
    features
).select(id_cols + feature_cols)

test_series = test_series.with_columns(
    features
).select(id_cols + feature_cols)

In [14]:
def make_train_dataset(train_data, train_events, drop_nulls=False) :
    
    series_ids = train_data['series_id'].unique(maintain_order=True).to_list()
    X, y = pl.DataFrame(), pl.DataFrame()
    for idx in tqdm(series_ids) : 
        
        # Normalizing sample features
        sample = train_data.filter(pl.col('series_id')==idx).with_columns(
            [(pl.col(col) / pl.col(col).std()).cast(pl.Float32) for col in feature_cols if col != 'hour']
        )
        
        events = train_events.filter(pl.col('series_id')==idx)
        
        if drop_nulls : 
            # Removing datapoints on dates where no data was recorded
            sample = sample.filter(
                pl.col('timestamp').dt.date().is_in(events['timestamp'].dt.date())
            )
        
        X = X.vstack(sample[id_cols + feature_cols])

        onsets = events.filter((pl.col('event') == 'onset') & (pl.col('step') != None))['step'].to_list()
        wakeups = events.filter((pl.col('event') == 'wakeup') & (pl.col('step') != None))['step'].to_list()

        # NOTE: This will break if there are event series without any recorded onsets or wakeups
        y = y.vstack(sample.with_columns(
            sum([(onset <= pl.col('step')) & (pl.col('step') <= wakeup) for onset, wakeup in zip(onsets, wakeups)]).cast(pl.Boolean).alias('asleep')
            ).select('asleep')
            )
    
    y = y.to_numpy().ravel()
    
    return X, y

In [15]:
def get_events(series, classifier) :
    '''
    Takes a time series and a classifier and returns a formatted submission dataframe.
    '''
    
    series_ids = series['series_id'].unique(maintain_order=True).to_list()
    events = pl.DataFrame(schema={'series_id':str, 'step':int, 'event':str, 'score':float})

    for idx in tqdm(series_ids) : 

        # Collecting sample and normalizing features
        scale_cols = [col for col in feature_cols if (col != 'hour') & (series[col].std() !=0)]
        X = series.filter(pl.col('series_id') == idx).select(id_cols + feature_cols).with_columns(
            [(pl.col(col) / series[col].std()).cast(pl.Float32) for col in scale_cols]
        )

        # Applying classifier to get predictions and scores
        preds, probs = classifier.predict(X[feature_cols]), classifier.predict_proba(X[feature_cols])[:, 1]

        #NOTE: Considered using rolling max to get sleep periods excluding <30 min interruptions, but ended up decreasing performance
        X = X.with_columns(
            pl.lit(preds).cast(pl.Int8).alias('prediction'), 
            pl.lit(probs).alias('probability')
                        )
        
        # Getting predicted onset and wakeup time steps
        pred_onsets = X.filter(X['prediction'].diff() > 0)['step'].to_list()
        pred_wakeups = X.filter(X['prediction'].diff() < 0)['step'].to_list()
        
        if len(pred_onsets) > 0 : 
            
            # Ensuring all predicted sleep periods begin and end
            if min(pred_wakeups) < min(pred_onsets) : 
                pred_wakeups = pred_wakeups[1:]

            if max(pred_onsets) > max(pred_wakeups) :
                pred_onsets = pred_onsets[:-1]

            # Keeping sleep periods longer than 30 minutes
            sleep_periods = [(onset, wakeup) for onset, wakeup in zip(pred_onsets, pred_wakeups) if wakeup - onset >= 12 * 30]

            for onset, wakeup in sleep_periods :
                # Scoring using mean probability over period
                score = X.filter((pl.col('step') >= onset) & (pl.col('step') <= wakeup))['probability'].mean()

                # Adding sleep event to dataframe
                events = events.vstack(pl.DataFrame().with_columns(
                    pl.Series([idx, idx]).alias('series_id'), 
                    pl.Series([onset, wakeup]).alias('step'),
                    pl.Series(['onset', 'wakeup']).alias('event'),
                    pl.Series([score, score]).alias('score')
                ))

    # Adding row id column
    events = events.to_pandas().reset_index().rename(columns={'index':'row_id'})

    return events

In [17]:

# from sklearn.model_selection import train_test_split

# train_ids, val_ids = train_test_split(series_ids, train_size=0.7, random_state=42)

# # We will collect datapoints at 10 minute intervals for training for validating
# train_data = train_series.filter(pl.col('series_id').is_in(train_ids)).take_every(12 * 10).collect()

# val_data = train_series.filter(pl.col('series_id').is_in(val_ids)).collect()
# val_solution = train_events.filter(pl.col('series_id').is_in(val_ids)).select(['series_id', 'event', 'step']).to_pandas()

# Collecting datapoints at every 5 minutes
train_data = train_series.filter(pl.col('series_id').is_in(series_ids)).take_every(12 * 5).collect()

KeyboardInterrupt: 

In [19]:
# Creating train dataset
X_train, y_train = make_train_dataset(train_data, train_events)

NameError: name 'train_data' is not defined